In [19]:
import json
import requests
import re
from datetime import datetime, timedelta


In [20]:

def login_silae_portal(username, password):
    session = requests.Session()
    
    login_url = "https://fiteco.rhsuite.silae.fr/login"
    
    # Première requête pour récupérer les cookies et tokens CSRF si nécessaire
    response = session.get(login_url)
    
    # Utiliser regex pour extraire le token CSRF au lieu de BeautifulSoup
    csrf_token = None
    csrf_pattern = r'name="_csrf_token"[^>]*value="([^"]*)"'
    match = re.search(csrf_pattern, response.text)
    if match:
        csrf_token = match.group(1)
    
    if not csrf_token:
        print("Token CSRF non trouvé")
        return None
    
    # Données de connexion avec le token CSRF
    login_data = {
        '_username': username,
        '_password': password,
        '_csrf_token': csrf_token
    }
    
    # Connexion
    response = session.post(login_url, data=login_data)
    
    if response.status_code == 200:
        print(f"✓ Connexion réussie")
        print(f"✓ PHPSESSID: {session.cookies.get('PHPSESSID')}")
        return session
    else:
        print(f"✗ Échec de la connexion: {response.status_code}")
        return None


    
def get_planning_events(session, date_from=None, date_to=None, view="timelineWeek"):
    """
    Récupère les événements du planning avec les headers complets
    """
    # Dates par défaut : semaine courante
    if not date_from:
        today = datetime.now()
        date_from = today.strftime("%Y-%m-%d")
    
    if not date_to:
        # Semaine actuelle (7 jours)
        date_to = (datetime.now() + timedelta(days=6)).strftime("%Y-%m-%d")
    
    # URL avec paramètres
    url = "https://fiteco.rhsuite.silae.fr/planning/json/employee/events"
    
    params = {
        'from': date_from,
        'to': date_to,
        'view': view  # timelineWeek, timelineDay, month, etc.
    }
    
    # Headers complets basés sur la capture réseau
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,fr;q=0.8,vi;q=0.7,fr-FR;q=0.6',
        'X-Requested-With': 'XMLHttpRequest',  # Important pour les requêtes AJAX
        'Referer': 'https://fiteco.rhsuite.silae.fr/planning/mon-planning',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0',
        'sec-ch-ua': '"Microsoft Edge";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin'
    }
    
    # Ajouter les headers à la session
    session.headers.update(headers)
    
    print(f"\n=== Récupération des événements ===")
    print(f"Période: {date_from} à {date_to}")
    print(f"Vue: {view}")
    
    # Faire la requête
    response = session.get(url, params=params)
    
    print(f"Status: {response.status_code}")
    print(f"URL complète: {response.url}")
    
    if response.status_code == 200:
        try:
            data = response.json()
            print(f"✓ Données récupérées: {len(data) if isinstance(data, list) else 'objet JSON'}")
            return data
        except json.JSONDecodeError:
            print("✗ Erreur de décodage JSON")
            print(f"Contenu: {response.text[:500]}")
            return None
    else:
        print(f"✗ Erreur: {response.status_code}")
        print(f"Réponse: {response.text[:500]}")
        return None



In [ ]:

def get_content(session, path="/"):
    url = f"https://fiteco.rhsuite.silae.fr{path}"  # ou /home
    response = session.get(url)
    
    if response.status_code == 200:
        # Retourner le texte HTML directement au lieu de BeautifulSoup
        return response.text
    else:
        print(f"Erreur: {response.status_code}")
        return None
    
def get_planning_resources(session):
    """
    Récupère les ressources de planning
    """
    url = "https://fiteco.rhsuite.silae.fr/planning/json/resources"
    
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://fiteco.rhsuite.silae.fr/planning/mon-planning'
    }
    
    session.headers.update(headers)
    response = session.get(url)
    
    if response.status_code == 200:
        try:
            return response.json()
        except json.JSONDecodeError:
            print("Erreur de décodage JSON pour les ressources")
            return None
    else:
        print(f"Erreur ressources: {response.status_code}")
        return None


In [14]:
# 1. Connexion
session = login_silae_portal("tylynn", "Tytony@2426#")

if session is None:
    raise SystemExit("Impossible de continuer sans connexion.")

# 2. Récupérer les événements de la semaine courante
events = get_planning_events(session)


# 3. Récupérer les ressources
resources = get_planning_resources(session)


✓ Connexion réussie
✓ PHPSESSID: ep2c2gk2ff6aludjm20b1iuh9e

=== Récupération des événements ===
Période: 2025-10-21 à 2025-10-27
Vue: timelineWeek
Status: 200
URL complète: https://fiteco.rhsuite.silae.fr/planning/json/employee/events?from=2025-10-21&to=2025-10-27&view=timelineWeek
✓ Données récupérées: 6
Status: 200
URL complète: https://fiteco.rhsuite.silae.fr/planning/json/employee/events?from=2025-10-21&to=2025-10-27&view=timelineWeek
✓ Données récupérées: 6


In [15]:
# # Vue mensuelle
events_month = get_planning_events(
    session,
    date_from="2025-10-01",
    date_to="2025-10-31",
    view="month"
)


=== Récupération des événements ===
Période: 2025-10-01 à 2025-10-31
Vue: month
Status: 200
URL complète: https://fiteco.rhsuite.silae.fr/planning/json/employee/events?from=2025-10-01&to=2025-10-31&view=month
✓ Données récupérées: 42
Status: 200
URL complète: https://fiteco.rhsuite.silae.fr/planning/json/employee/events?from=2025-10-01&to=2025-10-31&view=month
✓ Données récupérées: 42


In [8]:
def get_employee_map(resources):
    """Crée un mapping ID -> Nom d'employé"""
    employee_map = {}
    for res in resources:
        if res.get('type') == 'employee':
            employee_map[res.get('id')] = {
                'name': res.get('text'),
                'firstname': res.get('firstname'),
                'lastname': res.get('lastname'),
                'function': res.get('function', {}).get('label'),
                'weekHours': res.get('weekHours'),
                'contractStart': res.get('contractStart')
            }
    return employee_map

def find_employee_by_name(resources, search_name):
    """Trouve un employé par nom (recherche partielle, insensible à la casse)"""
    employee_map = get_employee_map(resources)
    search_name_lower = search_name.lower()

    matches = []
    for emp_id, emp_info in employee_map.items():
        if search_name_lower in emp_info['name'].lower():
            matches.append({'id': emp_id, **emp_info})

    return matches

def get_employee_shifts(events, employee_id, week_start=None, week_end=None):
    """Récupère les shifts d'un employé pour une période donnée"""
    # Convertir l'ID en string pour la comparaison
    employee_id_str = str(employee_id)

    # Filtrer les événements de cet employé
    employee_events = [
        e for e in events 
        if str(e.get('employee')) == employee_id_str
    ]

    # Filtrer par date si spécifié
    if week_start and week_end:
        filtered_events = []
        for event in employee_events:
            event_date_str = event.get('start', '').split()[0]
            if week_start <= event_date_str <= week_end:
                filtered_events.append(event)
        employee_events = filtered_events

    return employee_events

def format_shift(event):
    """Formate un événement en une représentation lisible"""
    event_type = event.get('type')
    label = event.get('label')
    code = event.get('code')
    start = event.get('start')
    end = event.get('end')
    duration = event.get('durationText')
    break_time = event.get('breakTime')

    # Parser la date et les heures
    if start:
        date_str = start.split()[0]
        time_start = start.split()[1]
    else:
        date_str = "N/A"
        time_start = "N/A"

    if end:
        time_end = end.split()[1]
    else:
        time_end = "N/A"

    result = {
        'date': date_str,
        'type': event_type,
        'label': label,
        'code': code,
        'start_time': time_start,
        'end_time': time_end,
        'duration': duration,
        'break_minutes': break_time
    }

    if break_time and break_time > 0:
        break_start = event.get('breakTimeStart', '')
        break_end = event.get('breakTimeEnd', '')
        if break_start:
            break_start = break_start.split()[1]
        if break_end:
            break_end = break_end.split()[1]
        result['break_start'] = break_start
        result['break_end'] = break_end

    return result

def calculate_total_hours(events):
    """Calcule le total d'heures travaillées"""
    work_events = [e for e in events if e.get('type') == 'WORK']

    total_minutes = 0
    for event in work_events:
        duration_text = event.get('durationText', '0h')
        if 'h' in duration_text:
            parts = duration_text.replace('h', ':').split(':')
            hours = int(parts[0]) if parts[0] else 0
            minutes = int(parts[1]) if len(parts) > 1 and parts[1] else 0
            total_minutes += hours * 60 + minutes

    total_hours = total_minutes // 60
    total_mins = total_minutes % 60

    return f"{total_hours}h{total_mins:02d}"

def display_employee_schedule(employee_name, events, resources):
    """Affiche le planning d'un employé"""
    # Trouver l'employé
    matches = find_employee_by_name(resources, employee_name)

    if not matches:
        print(f"❌ Aucun employé trouvé avec le nom '{employee_name}'")
        return

    if len(matches) > 1:
        print(f"⚠️  Plusieurs employés trouvés:")
        for idx, emp in enumerate(matches, 1):
            print(f"  {idx}. {emp['name']} (ID: {emp['id']})")
        print("\nVeuillez préciser votre recherche.")
        return

    # Un seul employé trouvé
    employee = matches[0]
    employee_id = employee['id']

    print(f"\n{'='*60}")
    print(f"📋 PLANNING DE {employee['name'].upper()}")
    print(f"{'='*60}")
    print(f"Fonction: {employee['function']}")
    print(f"Heures hebdomadaires: {employee['weekHours']}")
    print(f"Date d'embauche: {employee['contractStart']}")
    print()

    # Récupérer les shifts
    shifts = get_employee_shifts(events, employee_id)

    if not shifts:
        print("❌ Aucun shift trouvé pour cette période")
        return

    print(f"\n{'='*60}")
    print(f"📅 SHIFTS DE LA SEMAINE")
    print(f"{'='*60}\n")

    # Afficher chaque shift
    for event in shifts:
        shift = format_shift(event)

        if shift['type'] == 'WORK':
            print(f"📅 {shift['date']} - {shift['label']} ({shift['code']})")
            print(f"   ⏰ {shift['start_time']} → {shift['end_time']}")
            print(f"   ⌛ Durée: {shift['duration']}")
            if shift.get('break_minutes', 0) > 0:
                print(f"   ☕ Pause: {shift['break_minutes']} min ({shift.get('break_start')} - {shift.get('break_end')})")
        else:
            print(f"📅 {shift['date']} - {shift['label']} ({shift['code']})")
            print(f"   🏠 Repos/Absence")
        print()

    # Résumé
    work_shifts = [e for e in shifts if e.get('type') == 'WORK']
    absence_shifts = [e for e in shifts if e.get('type') == 'ABSENCE']

    print(f"\n{'='*60}")
    print(f"📊 RÉSUMÉ")
    print(f"{'='*60}")
    print(f"Jours travaillés: {len(work_shifts)}")
    print(f"Jours d'absence/repos: {len(absence_shifts)}")
    print(f"Total heures: {calculate_total_hours(shifts)}")
    print()


In [17]:
display_employee_schedule("Nguyen", events, resources)



📋 PLANNING DE TRAN LINH THUY NGUYEN
Fonction: 1er réceptionniste
Heures hebdomadaires: 35h
Date d'embauche: 2025-10-01


📅 SHIFTS DE LA SEMAINE

📅 2025-10-21 - Shift apres-midi (S-APM)
   ⏰ 15:20 → 23:10
   ⌛ Durée: 7h30
   ☕ Pause: 20 min (19:00 - 19:20)

📅 2025-10-22 - Shift matin (S-MAT)
   ⏰ 10:30 → 15:30
   ⌛ Durée: 4h40
   ☕ Pause: 20 min (12:00 - 12:20)

📅 2025-10-23 - Shift matin (S-MAT)
   ⏰ 08:30 → 16:30
   ⌛ Durée: 7h40
   ☕ Pause: 20 min (12:00 - 12:20)

📅 2025-10-24 - Repos (R)
   🏠 Repos/Absence

📅 2025-10-25 - Repos (R)
   🏠 Repos/Absence

📅 2025-10-26 - Shift matin (S-MAT)
   ⏰ 08:30 → 16:30
   ⌛ Durée: 7h40
   ☕ Pause: 20 min (12:00 - 12:20)


📊 RÉSUMÉ
Jours travaillés: 4
Jours d'absence/repos: 2
Total heures: 27h30

